In [27]:
#initial - import some basic libraries that might be useful here
%pylab inline
import pandas as pd
import seaborn as sns
pd.set_option('display.max_rows', 30)
import numpy as np
import pymongo

Populating the interactive namespace from numpy and matplotlib


D:\PythonSoftware\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['f']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [28]:
#connect to mongoDB
try:
    #use your database name, user and password here:
    #mongodb://<dbuser>:<dbpassword>@<mlab_url>/<database_name>
    with open("credentials1.txt", 'r', encoding='utf-8') as f:
        [name,password,url,dbname]=f.read().splitlines()
        conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
        print ("Connected successfully!!!")
except pymongo.errors.ConnectionFailure as e:
    print ("Could not connect to MongoDB: %s" % e) 

db = conn['aifinalproject_1']
collection1 = db['MSAI_fulltext1']
collection2 = db['MSAI_fulltext_1220']

Connected successfully!!!


In [29]:
#see how one collection element looks like
print(collection1.find_one())

{'_id': ObjectId('5a352ea10a411836947f6c28'), 'created_at': 'Fri Dec 15 19:18:49 +0000 2017', 'id': 941749403726970880, 'id_str': '941749403726970880', 'full_text': '#Microsoft adds more #AI smarts to #Office365 https://t.co/wjJUP2rJkY via @engadget #MicrosoftAI', 'truncated': False, 'display_text_range': [0, 96], 'entities': {'hashtags': [{'text': 'Microsoft', 'indices': [0, 10]}, {'text': 'AI', 'indices': [21, 24]}, {'text': 'Office365', 'indices': [35, 45]}, {'text': 'MicrosoftAI', 'indices': [84, 96]}], 'symbols': [], 'user_mentions': [{'screen_name': 'engadget', 'name': 'Engadget', 'id': 14372486, 'id_str': '14372486', 'indices': [74, 83]}], 'urls': [{'url': 'https://t.co/wjJUP2rJkY', 'expanded_url': 'https://www.engadget.com/2017/12/13/microsoft-adds-more-ai-smarts-office-365/', 'display_url': 'engadget.com/2017/12/13/mic…', 'indices': [46, 69]}]}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/android" rel="nofol

In [30]:
#read the csv and trying to create a table

#read the CSV file part 1
data = pd.read_csv("https://raw.githubusercontent.com/Nancyxw/AI_Final_Project/master/updated_fulltext.csv",index_col = 0)
df1 = pd.DataFrame(data)

#read the CSV file part 2
data = pd.read_csv("https://raw.githubusercontent.com/Nancyxw/AI_Final_Project/master/updated_fulltext2.csv",index_col = 0)
df2 = pd.DataFrame(data)

#merge the two dataframe
df = df1.merge(df2,how = 'outer')
clean_df = df[['created_at','display_text_range','full_text','favorite_count','retweeted','retweeted_status','retweet_count',]]

#update the text number
text_number = []
for i in clean_df['display_text_range']:
    text_number.append(i.split(',')[1][:-1])

clean_df['text_number'] = text_number
clean_df = clean_df.drop(['display_text_range'], axis = 1)
clean_df.head(2)

D:\PythonSoftware\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,created_at,full_text,favorite_count,retweeted,retweeted_status,retweet_count,text_number
0,Fri Dec 15 19:18:49 +0000 2017,#Microsoft adds more #AI smarts to #Office365 ...,0,False,NaN,0,96
1,Fri Dec 15 23:41:58 +0000 2017,RT @MSFTResearch: “As the first generation to ...,0,False,{'created_at': 'Wed Dec 13 22:25:52 +0000 2017...,136,140


In [31]:
#get all hashtags
allhashtags = []

collection_total = collection1.find()
collection_total2 = collection2.find()

for col in collection_total:
    for i in col['entities']['hashtags']:
        allhashtags.append(i['text'])

for col in collection_total2:
    for i in col['entities']['hashtags']:
        allhashtags.append(i['text'])

#see how it looks like
print(allhashtags[:10])

['Microsoft', 'AI', 'Office365', 'MicrosoftAI', 'AI', 'healthcare', 'healthcare', 'healthcare', 'Microsoft', 'healthcare']


In [32]:
#get all full text
alltext = []

collection_total = collection1.find()
collection_total2 = collection2.find()

for col in collection_total:
    alltext.append(col['full_text'])

for col in collection_total2:
    alltext.append(col['full_text'])

#see how it looks like
print(alltext[:2])

['#Microsoft adds more #AI smarts to #Office365 https://t.co/wjJUP2rJkY via @engadget #MicrosoftAI', 'RT @MSFTResearch: “As the first generation to live with #AI, we play a unique role in shaping the future and helping every person be part o…']


In [33]:
#see where are all users from
raw_location = []
location = []
collection_total = collection1.find()
collection_total2 = collection2.find()

for col in collection_total:
    raw_location.append(col['user']['location'])
    if col['user']['location'] != '':
        location.append(col['user']['location'])
        
for col in collection_total2:
    raw_location.append(col['user']['location'])
    if col['user']['location'] != '':
        location.append(col['user']['location'])

print(raw_location[:10])
print(location[:10])

['Dubai, UAE', '', 'Portugal', 'Madrid', 'Regina, SK', 'Milan, Italy', 'New York, NY', '', 'Trek Bicycle', 'Redmond, WA']
['Dubai, UAE', 'Portugal', 'Madrid', 'Regina, SK', 'Milan, Italy', 'New York, NY', 'Trek Bicycle', 'Redmond, WA', 'South Africa', 'HQ NYC/Menlo Park']


In [34]:
#merge the location in the dataframe
clean_df['location'] = raw_location
clean_df.head()

,created_at,full_text,favorite_count,retweeted,retweeted_status,retweet_count,text_number,location
0,Fri Dec 15 19:18:49 +0000 2017,#Microsoft adds more #AI smarts to #Office365 ...,0,False,NaN,0,96,"Dubai, UAE"
1,Fri Dec 15 23:41:58 +0000 2017,RT @MSFTResearch: “As the first generation to ...,0,False,{'created_at': 'Wed Dec 13 22:25:52 +0000 2017...,136,140,
2,Fri Dec 15 23:36:50 +0000 2017,RT @MSFTResearch: Microsoft’s Healthcare NExT ...,0,False,{'created_at': 'Thu Dec 14 01:04:16 +0000 2017...,27,140,Portugal
3,Fri Dec 15 23:34:27 +0000 2017,RT @MSFTResearch: Microsoft’s Healthcare NExT ...,0,False,{'created_at': 'Thu Dec 14 01:04:16 +0000 2017...,27,140,Madrid
4,Fri Dec 15 23:10:23 +0000 2017,RT @MSFTResearch: Microsoft’s Healthcare NExT ...,0,False,{'created_at': 'Thu Dec 14 01:04:16 +0000 2017...,27,140,"Regina, SK"


In [35]:
#get users' description
raw_description = []
description = []
collection_total = collection1.find()
collection_total2 = collection2.find()

for col in collection_total:
    raw_description.append(col['user']['description'])
    if col['user']['description'] != '':
        description.append(col['user']['description'])
        
for col in collection_total2:
    raw_description.append(col['user']['description'])
    if col['user']['description'] != '':
        description.append(col['user']['description'])

print(raw_description[:2])
print(description[:2])

["Programme Coordinator, School of Media, Murdoch University Dubai, UAE (@MurdochUniDubai). Let's talk #Journalism #Diaspora #MediaStudies. RT ≠ endorsement.", "I'm a creative marketing producer/director, constant work in progress, pretty decent wife, proud mother, and just happy to be alive."]
["Programme Coordinator, School of Media, Murdoch University Dubai, UAE (@MurdochUniDubai). Let's talk #Journalism #Diaspora #MediaStudies. RT ≠ endorsement.", "I'm a creative marketing producer/director, constant work in progress, pretty decent wife, proud mother, and just happy to be alive."]


In [36]:
#get 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count' & 'statuses_count'
raw_followers_count = []
raw_friends_count = []
raw_listed_count = []
raw_created_at = []
raw_favourites_count = []
raw_statuses_count = []

collection_total = collection1.find()
collection_total2 = collection2.find()

for col in collection_total:
    raw_followers_count.append(col['user']['followers_count'])
    raw_friends_count.append(col['user']['friends_count'])
    raw_listed_count.append(col['user']['listed_count'])
    raw_created_at.append(col['user']['created_at'])
    raw_favourites_count.append(col['user']['favourites_count'])
    raw_statuses_count.append(col['user']['statuses_count'])
        
for col in collection_total2:
    raw_followers_count.append(col['user']['followers_count'])
    raw_friends_count.append(col['user']['friends_count'])
    raw_listed_count.append(col['user']['listed_count'])
    raw_created_at.append(col['user']['created_at'])
    raw_favourites_count.append(col['user']['favourites_count'])
    raw_statuses_count.append(col['user']['statuses_count'])
    
print(raw_followers_count[:2],
raw_friends_count[:2],
raw_listed_count[:2],
raw_created_at[:2],
raw_favourites_count[:2],
raw_statuses_count[:2])

[1075, 165] [2014, 331] [70, 11] ['Mon Mar 01 20:26:12 +0000 2010', 'Thu Nov 11 13:54:11 +0000 2010'] [3086, 880] [12535, 544]


In [37]:
#add users' detail to the dataframe
clean_df['user_followers'] = raw_followers_count
clean_df['user_friends'] = raw_friends_count
clean_df['user_listed'] = raw_listed_count
clean_df['user_created_at'] = raw_created_at
clean_df['user_favourites_count'] = raw_favourites_count
clean_df['user_statuses_number'] = raw_statuses_count

clean_df.head(2)

,created_at,full_text,favorite_count,retweeted,retweeted_status,retweet_count,text_number,location,user_followers,user_friends,user_listed,user_created_at,user_favourites_count,user_statuses_number
0,Fri Dec 15 19:18:49 +0000 2017,#Microsoft adds more #AI smarts to #Office365 ...,0,False,NaN,0,96,"Dubai, UAE",1075,2014,70,Mon Mar 01 20:26:12 +0000 2010,3086,12535
1,Fri Dec 15 23:41:58 +0000 2017,RT @MSFTResearch: “As the first generation to ...,0,False,{'created_at': 'Wed Dec 13 22:25:52 +0000 2017...,136,140,,165,331,11,Thu Nov 11 13:54:11 +0000 2010,880,544


In [42]:
#check retweet status and make it '1' if it's a retweet, '0' if it's not
retweet_yesorno = []
for text in alltext:
    if 'RT ' in text:
        retweet_yesorno.append(1)
    else:
        retweet_yesorno.append(0)

In [47]:
#add this to the dataframe
clean_df['Is_this_a_retweet'] = retweet_yesorno
clean_df.head()

,created_at,full_text,favorite_count,retweeted,retweeted_status,retweet_count,text_number,location,user_followers,user_friends,user_listed,user_created_at,user_favourites_count,user_statuses_number,Is_this_a_retweet
0,Fri Dec 15 19:18:49 +0000 2017,#Microsoft adds more #AI smarts to #Office365 ...,0,False,NaN,0,96,"Dubai, UAE",1075,2014,70,Mon Mar 01 20:26:12 +0000 2010,3086,12535,0
1,Fri Dec 15 23:41:58 +0000 2017,RT @MSFTResearch: “As the first generation to ...,0,False,{'created_at': 'Wed Dec 13 22:25:52 +0000 2017...,136,140,,165,331,11,Thu Nov 11 13:54:11 +0000 2010,880,544,1
2,Fri Dec 15 23:36:50 +0000 2017,RT @MSFTResearch: Microsoft’s Healthcare NExT ...,0,False,{'created_at': 'Thu Dec 14 01:04:16 +0000 2017...,27,140,Portugal,334,190,457,Wed May 27 22:16:44 +0000 2009,12396,21113,1
3,Fri Dec 15 23:34:27 +0000 2017,RT @MSFTResearch: Microsoft’s Healthcare NExT ...,0,False,{'created_at': 'Thu Dec 14 01:04:16 +0000 2017...,27,140,Madrid,3258,204,143,Sun Sep 20 20:19:29 +0000 2009,2600,1554,1
4,Fri Dec 15 23:10:23 +0000 2017,RT @MSFTResearch: Microsoft’s Healthcare NExT ...,0,False,{'created_at': 'Thu Dec 14 01:04:16 +0000 2017...,27,140,"Regina, SK",1360,1989,212,Fri May 09 00:34:39 +0000 2008,1628,6748,1
